In [ ]:
from pathlib import Path

import folium
import pandas as pd
import geopandas as gpd
import contextily as ctx

In [ ]:
coverage_dir = Path(r"Q:\CMP\LOS Monitoring 2023\transit\coverage\2023")
# headways = ["10min", "5min", "all"]
# headways: {filename: label name on map}
# the order of headways matter, because I'm using a groupby().first() later
# to only keep the first entry for each MAZ, since I'm plotting a chloropleth.
# (`5min` is a subset of `10min` is a subset of `all`, so I'm just keeping the
#  most specific label)
headways = {
    "5min": "5-min headway",
    "10min": "10-min headway",
    "all": "any service",
}
time_periods = ["AM", "PM"]
year = 2023

In [ ]:
def construct_period_gdf(gdfs, sort_by_headway=True):
    # keep only the first entry for each MAZ, because `5min` is a subset of
    # `10min` is a subset of `all`, so I'm just keeping the most specific label,
    # for plotting a chloropleth later
    df = pd.concat(gdfs).groupby("MAZID").first()
    df["headway"].astype("category").cat.set_categories(
        headways.values(), ordered=True
    )
    if sort_by_headway:
        df = df.sort_values("headway")
        # TODO I'm trying to pre-sort to get the plot legend to show up as
        # 5, 10, any rather than any other order
    return gpd.GeoDataFrame(df).set_crs(get_crs(gdfs))


def get_crs(gdfs):
    # pull out the crs from each of the gdf's that are read in, making sure
    # they're the same
    crs = set(gdf.crs for gdf in gdfs)
    assert (
        len(crs) == 1
    ), "The crs isn't all the same for the different input gpkg's"
    return crs.pop()

In [ ]:
for time_period in time_periods:
    gdfs = []
    for headway in headways.keys():
        filepath = (
            coverage_dir
            / headway
            / f"coverage_{headway}_{year}_{time_period}buffer.gpkg"
        )
        gdf = gpd.read_file(filepath)
        gdf["headway"] = headways[headway]
        gdfs.append(gdf)
    period_gdf = construct_period_gdf(gdfs, sort_by_headway=True)

In [ ]:
# 'Web map tiles are typically provided in Web Mercator (EPSG 3857)' (GeoPandas docs)
ax = (
    period_gdf.sort_values("headway")
    .to_crs(epsg=3857)
    .plot(
        column="headway",
        legend=True,
        legend_kwds={"loc": "upper left"},
        figsize=(10, 10),
    )
)
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.set_axis_off()

In [ ]:
for time_period in time_periods:
    # START FUNC HERE
    map = folium.Map(
        location=[37.76, -122.44],
        tiles="CartoDB Positron",
        zoom_start=12,
        control_scale=True,
    )
    for headway in headways.keys():
        filepath = (
            coverage_dir
            / headway
            / f"coverage_{headway}_{year}_{time_period}buffer.gpkg"
        )
        gdf = gpd.read_file(filepath)
        folium.GeoJson(
            data=gdf  # , style_function=lambda x: {"fillColor": "orange"}
        ).add_to(map)
map